In [1]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns
from importlib import reload
import matplotlib.pyplot as plt
import preprocess_data 

reload(preprocess_data)

%matplotlib inline

In [2]:
inpatient_specialty = pd.read_csv('/data/public/MLA/VCHAMPS-Quality/inpatient_specialty_qual.csv').iloc[:,1:]

In [3]:
inpatient_specialty['Specialty start date'] = inpatient_specialty['Specialty start date'].apply(preprocess_data.preprocess_time_string)
inpatient_specialty['Specialty end date'] = inpatient_specialty['Specialty end date'].apply(preprocess_data.preprocess_time_string)
inpatient_specialty['period'] = preprocess_data.get_period(inpatient_specialty['Specialty start date'],inpatient_specialty['Specialty end date'])

In [4]:
specialty_class = preprocess_data.generate_dict_from_csv('/home/vivi/FDA_datasets/specialty_class.csv','Specialty','Class')

In [5]:
inpatient_specialty['Specialty encoded'] = preprocess_data.preprocess_substitute_categories(specialty_class, 
                                                                               inpatient_specialty['Specialty'])

In [7]:
path = '/home/hassan/lily/MLA/FDA/inpatient_simple_quality.csv'
inpatient = pd.read_csv(path).iloc[:,1:]
inpatient.head()

,Internalpatientid,num_stays,stay_length,num_unique_units,num_transfers,num_cvd_admission,CVD,unique_admitting_specialty,unique_discharging_specialty,DOMICILIARY,...,freq,Medical,Mental,Others_Specialty,Rehab,Gerontology,CVD_readmission,next_readmission_time,Discharge date,readmission within 300 days
0,67,1,0.18,1,0,0,0,1,1,0,...,1.00,1,0,0,0,0,0,0.00,2019-04-29 22:39:20.0,0
1,200,4,17.90,2,0,0,0,4,3,0,...,0.25,1,0,0,2,0,0,130.54,2019-01-31 22:59:02.0,1
2,291,7,79.68,2,0,3,1,3,2,0,...,2.33,0,0,0,7,0,0,-6.93,2006-07-08 13:56:24.0,1
3,330,3,17.38,1,0,1,1,2,2,0,...,3.00,1,0,0,2,0,0,993.85,2019-06-28 14:50:58.0,0
4,351,3,22.14,1,0,3,1,2,2,0,...,3.00,0,2,0,0,1,1,77.26,2023-07-10 21:33:42.0,1


In [8]:
inpatient_date = inpatient[['Internalpatientid','Discharge date']]
inpatient_date = inpatient_date.drop_duplicates(keep = 'first')
inpatient_date

,Internalpatientid,Discharge date
0,67,2019-04-29 22:39:20.0
1,200,2019-01-31 22:59:02.0
2,291,2006-07-08 13:56:24.0
3,330,2019-06-28 14:50:58.0
4,351,2023-07-10 21:33:42.0
...,...,...
627,166881,2001-01-09 21:07:54.0
628,167102,2011-11-27 12:57:02.0
629,167404,2004-01-30 11:12:01.0
630,167917,2016-08-07 11:25:36.0


In [9]:
inpatient_specialty2 = inpatient_specialty.merge(inpatient_date, how = 'left', on = 'Internalpatientid')
inpatient_specialty3 = inpatient_specialty2.loc[inpatient_specialty2['Specialty end date'] <= inpatient_specialty2['Discharge date']]
inpatient_specialty3

,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty,State,period,Specialty encoded,Discharge date
0,100399,84.470360,2008-12-27 22:00:08,2009-01-01 01:41:45,"PULMONARY, NON-TB",Minnesota,4 days 03:41:37,NaN,2010-11-09 15:00:47.0
1,100399,85.551411,2010-01-27 00:58:53,2010-02-03 20:55:56,SUBSTANCE ABUSE TRMT UNIT,Minnesota,7 days 19:57:03,1.0,2010-11-09 15:00:47.0
3,100694,72.493945,2004-08-04 12:53:09,2004-08-07 14:21:19,SURGICAL STEPDOWN,Idaho,3 days 01:28:10,0.0,2020-09-22 11:36:18.0
4,102468,59.676888,2011-04-08 21:39:44,2011-04-12 23:58:20,GEM ACUTE MEDICINE,North Carolina,4 days 02:18:36,0.0,2016-03-20 22:52:41.0
5,102468,64.621002,2016-03-19 23:25:53,2016-03-20 22:13:36,GEM ACUTE MEDICINE,North Carolina,0 days 22:47:43,0.0,2016-03-20 22:52:41.0
...,...,...,...,...,...,...,...,...,...
11384,98416,73.041255,2000-09-25 13:53:23,2000-09-26 05:09:15,HALFWAY HOUSE,California,0 days 15:15:52,1.0,2005-12-11 15:50:53.0
11385,98416,78.018555,2005-09-18 18:45:38,2005-09-18 18:45:38,CARDIOLOGY,California,0 days 00:00:00,0.0,2005-12-11 15:50:53.0
11386,99024,68.288449,2014-08-25 10:16:36,2014-08-25 10:16:36,GENERAL(ACUTE MEDICINE),Oregon,0 days 00:00:00,0.0,2015-01-27 03:32:24.0
11387,9929,72.785598,2011-11-05 03:15:59,2011-11-07 03:15:58,ANESTHESIOLOGY,Arizona,1 days 23:59:59,0.0,2012-11-25 01:07:41.0


In [10]:
inpatient_specialty3['specialty count'] = inpatient_specialty3['Specialty']\
    .groupby(inpatient_specialty3['Internalpatientid']).transform('count')

specialty_medical_count = inpatient_specialty3.groupby('Internalpatientid')['Specialty encoded']\
    .apply(lambda x: (x == 0).sum()).reset_index(name='specialty medical count')

specialty_support_count = inpatient_specialty3.groupby('Internalpatientid')['Specialty encoded']\
    .apply(lambda x: (x == 1).sum()).reset_index(name='specialty support count')

inpatient_specialty3['mean age at specailty'] = inpatient_specialty3['Age at specialty']\
    .groupby(inpatient_specialty3['Internalpatientid']).transform('mean')

/tmp/ipykernel_256535/3919990082.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inpatient_specialty3['specialty count'] = inpatient_specialty3['Specialty']\
/tmp/ipykernel_256535/3919990082.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inpatient_specialty3['mean age at specailty'] = inpatient_specialty3['Age at specialty']\


In [11]:
period_mean = inpatient_specialty.groupby('Internalpatientid')['period'].agg(lambda x: x).reset_index()
period_mean['period'] = period_mean['period'].apply(lambda x: np.nanmean(x.total_seconds()/86400))

period_std = inpatient_specialty.groupby('Internalpatientid')['period'].agg(lambda x: x).reset_index()
period_std['period'] = period_std['period'].apply(lambda x: np.nanstd(x.total_seconds()/86400,ddof=0))

period_mean = period_mean.rename(columns={'period': 'period mean'})
period_std = period_std.rename(columns={'period': 'period std'})

In [12]:
inpatient_specialty4 = inpatient_specialty3[['Internalpatientid', 'mean age at specailty','specialty count']].sort_values('Internalpatientid')
inpatient_specialty4 = inpatient_specialty4.groupby('Internalpatientid').apply(lambda x:x)
inpatient_specialty4 = inpatient_specialty4.drop_duplicates().reset_index(drop = True)
inpatient_specialty4

/tmp/ipykernel_256535/554132651.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  inpatient_specialty4 = inpatient_specialty4.groupby('Internalpatientid').apply(lambda x:x)


,Internalpatientid,mean age at specailty,specialty count
0,67,56.997988,1
1,200,78.545544,12
2,291,82.644124,23
3,330,69.705393,7
4,351,84.736257,11
...,...,...,...
627,166881,74.958584,1
628,167102,64.931204,6
629,167404,77.965193,3
630,167917,44.730726,12


In [13]:
inpatient_specialty5 = inpatient_specialty4.merge(period_mean, how = 'left', on = 'Internalpatientid')
inpatient_specialty5 = inpatient_specialty5.merge(period_std, how = 'left', on = 'Internalpatientid')

inpatient_specialty5 = inpatient_specialty5.merge(specialty_medical_count, how = 'left', on = 'Internalpatientid')
inpatient_specialty5 = inpatient_specialty5.merge(specialty_support_count, how = 'left', on = 'Internalpatientid')

inpatient_specialty5 = inpatient_specialty5.dropna(subset = 'period mean')
inpatient_specialty5

,Internalpatientid,mean age at specailty,specialty count,period mean,period std,specialty medical count,specialty support count
0,67,56.997988,1,0.089728,0.089728,1,0
1,200,78.545544,12,1.771890,3.276867,7,5
2,291,82.644124,23,3.711998,5.827537,7,15
3,330,69.705393,7,1.829252,1.986030,7,0
4,351,84.736257,11,2.304762,2.520672,3,8
...,...,...,...,...,...,...,...
627,166881,74.958584,1,1.937271,2.161738,1,0
628,167102,64.931204,6,1.591543,1.560624,5,1
629,167404,77.965193,3,3.875710,3.145414,2,0
630,167917,44.730726,12,0.803098,0.899967,12,0


In [14]:
inpatient_specialty5.to_csv('/home/vivi/FDA_datasets/inpatient_specialty_quality_admission.csv')